# Motivating example: Parallel image processing with scikit-image

To get a sense of what IPython.parallel might be used for,
we start with an example of some batch processing of image files with [scikit-image](http://scikit-image.org/).
We will revisit pieces of this example as we learn about the different components of IPython.

You can download images with [this notebook](../download-images.ipynb), or get a zip [here](https://s3.amazonaws.com/ipython-parallel-data/images.zip), or find any images on your computer.




In [ ]:
import sys
import requests
from zipfile import ZipFile, BadZipFile
from IPython.html.widgets import IntProgress
from IPython.display import display

images = os.path.join('..', 'images')
images_url = "https://s3.amazonaws.com/ipython-parallel-data/images.zip"


def download_images():
    r = requests.get(images_url, stream=True)
    content_length = r.headers.get('content-length')
    print("Downloading images")
    sys.stdout.flush()
    p = IntProgress(max=content_length)
    display(p)
    with open(images_zip, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            p.value += len(chunk)
            f.write(chunk)


if not os.path.exists(images):
    images_zip = images + '.zip'
    if os.path.exists(images_zip):
        try:
            zf = ZipFile(images_zip)
        except BadZipFile:
            os.remove(images_zip)
        else:
            zf.close()
    if not os.path.exists(images_zip):
        download_images()
    
    ZipFile(images_zip).extractall('..')


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import sys,os,re,time
import urllib

import numpy as np

from IPython import parallel

In [ ]:
from skimage.io import imread
from skimage.feature import corner_harris, corner_peaks

Define a function to find the corners of the image:

In [ ]:
def find_corners(path, min_distance=5):
    """Find corners in an image at path
    
    Returns the image and the corner lists.
    """
    img = imread(path, flatten=True)
    corners = corner_peaks(corner_harris(img), min_distance=min_distance)
    return img, corners


and show the contours on the image with matplotlib:

In [ ]:
def plot_corners(img, corners, show=True):
    """Display the image and plot all contours found"""
    plt.imshow(img, cmap='gray')
    plt.plot(corners[:,1], corners[:,0], 'r+', markeredgewidth=1.5, markersize=8) # Plot corners
    plt.axis('image')
    plt.xticks([])
    plt.yticks([])
    if show:
        plt.show()


In [ ]:
def get_corners_image(path):
    """Given a path, return a PNG of the image with contour lines
    
    Calls both find_contours and plot_contours
    """
    from IPython.core.pylabtools import print_figure
    
    img, corners = find_corners(path)
    plot_corners(img, corners, show=False)
    fig = plt.gcf()
    pngdata = print_figure(fig)
    plt.close(fig)
    return pngdata


In [ ]:
import os
pictures_dir = os.path.join('..', 'images', 'portrait')

pictures = []
for directory, subdirs, files in os.walk(pictures_dir):
    for fname in files:
        if fname.lower().endswith(('.jpg', '.png')):
            pictures.append(os.path.join(directory, fname))

pictures[:5]

Let's test our function locally, to see what it does.

In [ ]:
for p in pictures[:3]:
    img, corners = find_corners(p)
    plot_corners(img, corners)


# Now in parallel

First, we connect our parallel Client

In [ ]:
rc = parallel.Client()
all_engines = rc[:]
view = rc.load_balanced_view()

Then we initialize the namespace on all of the engines with imports

In [ ]:
%px import os; os.chdir("{os.getcwd()}")

In [ ]:
%%px
%matplotlib inline
import matplotlib.pyplot as plt

from skimage.io import imread
from skimage.feature import corner_harris, corner_peaks

and make sure some functions are defined everywhere (this is only necessary for the `contours_in_url` case)

In [ ]:
all_engines.push(dict(
    plot_corners=plot_corners,
    find_corners=find_corners,
))

Now we can iterate through all of our pictures, and detect and display any corners we find

In [ ]:
from IPython.display import display, Image

amr = view.map_async(get_corners_image, pictures[:20], ordered=False)
for pngdata in amr:
    display(Image(data=pngdata))
